In [ ]:
# PYTHON=/projects/CIBCIGroup/00DataUploading/Xiaowei/anaconda/dl_pl/dl_pl/bin/python

In [ ]:
# from tqdm import tqdm
from shallowmind.src.model import ModelInterface
from shallowmind.src.data import DataInterface
from shallowmind.api.infer import prepare_inference
import pandas as pd
import torch
from tqdm import tqdm
import pickle
import numpy as np
import pytorch_lightning as pl
pl.seed_everything(42)
from einops import rearrange
import os
import numpy as np
from umap.umap_ import UMAP
import matplotlib.pyplot as plt






In [ ]:
save_dir='/projects/CIBCIGroup/00DataUploading/Liang/fuzzy_fnirs/Friends_fNIRS_Fuzzy/'
os.makedirs(save_dir, exist_ok=True)

ckpt = '/projects/CIBCIGroup/00DataUploading/Liang/fuzzy_fnirs/best_models/fuzzy/FTF-ds=PictureRating_l-d=3-nh=11-r=1-bs=256-ls=0.00015-label=hand/ckpts/exp_name=FTF-ds=PictureRating_l-d=3-nh=11-r=1-bs=256-ls=0.00015-label=hand-cfg=FuzzyTramsformer_num_rules1_num_heads11_dataset_name=PictureRating_label=hand_base_lr=1.5e-4_depth=3_batch_size=256-bs=256-seed=42-val_f1_score=0.6537.ckpt'
config = 'best_models/fuzzy/FTF-ds=PictureRating_l-d=3-nh=11-r=1-bs=256-ls=0.00015-label=hand/FuzzyTramsformer_num_rules1_num_heads11_dataset_name=PictureRating_label=hand_base_lr=1.5e-4_depth=3_batch_size=256.py'



data_module, model = prepare_inference(config, ckpt)
data_module.setup()

# 1. draw centers
# 2. Friend Attention statistics


centers=model.model.encoder.encoder.layers[0].self_attn.rules_keys

centers=rearrange(centers, 'h r d -> r (h d)')
test_loader = data_module.test_dataloader()
data_table = test_loader.dataset.data_table
model = model.eval()


device = torch.device('cpu')
model = model.to(device)
val_loader = data_module.val_dataloader()
data_table = val_loader.dataset.data_table
res = pd.DataFrame(columns=['pred', 'label'])

embs = []
data_info = test_loader.dataset.data_index_table
all_data=[]
for batch_idx, d in tqdm(enumerate(test_loader), total=len(test_loader)):
    torch.cuda.empty_cache()
    data = {'seq': d[0]['seq'].to(device),}
    # all_data.append(data['seq'])
    label = d[1].to(device)
    with torch.no_grad():
        imgs = model.model.gen_data(data)
        # print(model.model.encoder.encoder.layers[0].self_attn)
        output,query= model.model.encoder.encoder.layers[0].self_attn(imgs,imgs,imgs,return_query=True)
        all_data.append(query)
        latent = model.model.forward_encoder(imgs)
        
        latent = torch.cat([latent[:latent.shape[0]//2, :], latent[latent.shape[0]//2:, :]], dim=1)
        embs.append(latent.cpu().numpy())
        latent = model.model.cls(latent)
        pred = latent.squeeze(1)
        pred = torch.argmax(pred, dim=1)
        pred = pred.cpu().numpy()
        label = label.cpu().numpy()
        res = pd.concat([res, pd.DataFrame({'pred': pred, 'label': label})], axis=0)
